## Introduction
This notebook evaluates the T5-small model's performance on question answering tasks 
We analyze:
- Zero-shot capabilities
- Few-shot learning
- Context vs no-context comparisons
- Fine-tuned performance 

## 1. Dataset Loading and Utility Functions

In [6]:
import pandas as pd
finetuning_data=pd.read_parquet("/kaggle/input/nlp-resources/tuning_data.parquet")
hyperparameter_tuning_data=pd.read_parquet("/kaggle/input/nlp-resources/hyperparameter_tuning_data.parquet")
test_data=pd.read_parquet("/kaggle/input/nlp-resources/test_data.parquet")
contexts=pd.concat([test_data["context"],hyperparameter_tuning_data["context"],finetuning_data["context"]]).to_list()


In [3]:
import torch
import pandas as pd
from transformers import T5Tokenizer
from datasets import load_dataset
from datasets import Dataset
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import EarlyStoppingCallback
from transformers import TrainerCallback
from functools import reduce
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics.pairwise import cosine_similarity
import string
import re
from collections import Counter
import numpy as np
nltk.download('punkt')

def preprocess(example,tokenizer,max_length_input=512,max_length_labels=64,no_context=False):
    input_text = f"question: {example['question']}" 
    if (not no_context):
        input_text+=f" context: {example['context']}"
    target_text = example["answer"]
    inputs = tokenizer(input_text, max_length=max_length_input, truncation=True, padding="max_length")
    labels = tokenizer(target_text, max_length=max_length_labels, truncation=True, padding="max_length")["input_ids"] 
    inputs["labels"] =labels
    return inputs

def generate_answer(model, tokenizer, question, device, max_input_length=512, max_output_length=64, context=None):
    model.eval()
    input_text = f"question: {question}"
    if(context):
        input_text+= f" context: {context}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=max_input_length, truncation=True)
    inputs = inputs.to(device)

    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_output_length,
            num_beams=4,
            early_stopping=True,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.9,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3
        
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def normalize_answer(s):
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def remove_punc(text):
        return ''.join(ch for ch in text if ch not in set(string.punctuation))
    def white_space_fix(text):
        return ' '.join(text.split())
    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def exact_match_score(prediction, ground_truth):
    return int(normalize_answer(prediction) == normalize_answer(ground_truth))

def f1_score(prediction, ground_truth):
    pred_tokens = normalize_answer(prediction).split()
    gt_tokens = normalize_answer(ground_truth).split()
    common = Counter(pred_tokens) & Counter(gt_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(gt_tokens)
    return 2 * precision * recall / (precision + recall)
    
def bleu_score(prediction, ground_truth):
    smoothie = SmoothingFunction().method4  # migliora BLEU su frasi corte
    pred_tokens = nltk.word_tokenize(prediction.lower())
    gt_tokens = nltk.word_tokenize(ground_truth.lower())
    return sentence_bleu([gt_tokens], pred_tokens, smoothing_function=smoothie)
    
def get_top_k_contexts(context_embeddings,model_embed,query,k=3):
    query_emb=model_embed.encode([query], convert_to_numpy=True)
    similarity=cosine_similarity(query_emb,context_embeddings)[0]
    #print(similarity)
    top_k_idx = similarity.argsort()[-k:]
    return [contexts[i] for i in top_k_idx]
    
def eval_answers(model,tokenizer,test_data=test_data,max_input=512,max_output=64, rag=False, no_context=False):
    exact_matches = []
    f1_scores = []
    bleu_scores=[]
    for idx, row in test_data.iterrows():
        context= None if no_context else row['context']
        pred_answer = generate_answer(model, tokenizer, row['question'],max_input_length=max_input,max_output_length=max_output, device=device,context=context )
        em = exact_match_score(pred_answer, row['answer'])
        f1 = f1_score(pred_answer, row['answer'])
        bleu = bleu_score(pred_answer, row['answer'])
        exact_matches.append(em)
        f1_scores.append(f1)
        bleu_scores.append(bleu)
    return sum(bleu_scores) / len(bleu_scores) * 100,sum(exact_matches) / len(exact_matches) * 100, sum(f1_scores) / len(f1_scores) * 100
def print_scores(avg_bleu=-1, avg_em=-1, avg_f1=-1,model_name="",scores_path="/kaggle/input/nlp-resources/scores_t5_rag.parquet",use_backup=False):
    
    if(use_backup):
        scores=pd.read_parquet(scores_path)
        scores_array=scores[scores["model_name"]==model_name][["avg_bleu","avg_em","avg_f1"]].values[0]
    else:
        scores_array=np.array([avg_bleu, avg_em, avg_f1])
    print(f"Scores of {model_name} model :")
    print(f"Average BLEU score: {scores_array[0]:.2f}")
    print(f"Exact Match: {scores_array[1]:.2f}%")
    print(f"F1 Score: {scores_array[2]:.2f}%")
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

2025-05-24 00:29:23.378200: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748046563.547707      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748046563.602669      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 2. Evaluation Metrics

We use three key metrics:

| **Metric**       | **Description**         | **Importance**                |
|------------------|-------------------------|-------------------------------|
| Exact Match (EM) | Strict string matching  | Measures precision            |
| F1 Score         | Token-level overlap     | Balances precision/recall     |
| BLEU Score       | N-gram similarity       | Evaluates fluency             |


## 3. Zero-Shot Evaluation

In [4]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq

tokenizer_T5 = T5Tokenizer.from_pretrained("t5-small")
model_T5_pretrained = T5ForConditionalGeneration.from_pretrained("t5-small")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_T5_pretrained.to(device)
model_T5_pretrained.eval()

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

#### Zero shot sample :

In [5]:
sample_zero_shot=test_data.sample(n=1,random_state=42)
print(f"Sampled exapmle:\n#Question: {sample_zero_shot['question'].values[0]}\n#Context:{sample_zero_shot['context'].values[0]}")

Sampled exapmle:
#Question: What are the responsibilities and requirements for the Software Architect position at the entertainment company?
#Context:an entertainment company specializing in advice-based products and services is looking for a software architect to work alongside their growing team, as a contractor. responsibilities and requirements -develop and analyze the design and architecture of complex software application systems. -provide architectural and implementation oversight and guidance to ensure consistency and quality of design and code. -analyze and document existing systems, review preexisting complex code and provide recommendations to improve performance & maintainability. -ability to communicate technical issues and concepts clearly, both orally and in writing -write test and debug complex problems in various modules of the various software application -code reviews -manage test and acceptance activities -direct contribution to development and test efforts -manage 

#### Zero shot generated output :

In [6]:
zero_shot_answer=generate_answer(model_T5_pretrained, tokenizer_T5, question=sample_zero_shot['question'].values[0], device=device, max_input_length=512, max_output_length=128, context=sample_zero_shot['context'].values[0])
print(f"Genereated answer:{zero_shot_answer}\n\nDataset Answer:{sample_zero_shot['answer'].values[0]}")
print(f"\nComparison with dataset answer:\nF1:{f1_score(zero_shot_answer, sample_zero_shot['answer'].values[0]):.2f}",f"BLEU:{bleu_score(zero_shot_answer, sample_zero_shot['answer'].values[0]):.2f}",f"EM:{exact_match_score(zero_shot_answer, sample_zero_shot['answer'].values[0]):.2f}")

Genereated answer:develop and analyze the design and architecture of complex software applications

Dataset Answer:The responsibilities and requirements for the Software Architect position include developing and analyzing the design and architecture of complex software application systems, providing architectural and implementation oversight, analyzing and documenting existing systems, communicating technical issues clearly, writing and debugging complex problems in various software modules, managing test and acceptance activities, designing and building reusable modules, writing application code using latest C# and ASP.NET framework, assisting in building database structure for SQL Server, understanding cloud engineering, handling and prioritizing multiple tasks, demonstrating strong analytical and problem solving skills, having very good communication skills, having 10+ years of professional web development experience using C#, .NET, ASP.NET 3.5 – 4.0, MVC, and WCF, having 7+ year’s 

## 4. Few-Shot Prompt (1-Shot)

We manually construct a prompt that includes one example (1-shot learning). This tests the model's ability to learn from a single example.


In [7]:
test_data["len"]=test_data["context"].apply(len)
samples=test_data.sort_values(by="len").head(2)
shots=samples.head(1)
test_sample=samples.iloc[-1]
promt=""
index=1
for i,row in shots.iterrows():
    example=f"Example {index} question: {row['question']} context: {row['context']} answer: {row['answer']}\n"
    promt+=example
    index+=1
promt+=f"question: {test_sample['question']} context: {test_sample['context']} answer:"
print(promt)

Example 1 question: Who is the speaker for the lecture on Numerical Characterization of Powder Flowability? context: 【学术活动预告】numerical characterization of powder flowability/粉体流动性的计算模拟研究 报告题目：numerical characterization of powder flowability/粉体流动性的计算模拟研究 报告人：裴春雷 博士 英国剑桥大学材料科学及冶金系博士后 时 间：2018年1月2日（周二）下午15:30 地 点：紫金校区能源学院学术报告厅（一教237） 邀请人：南京师范大学能源与机械工程学院、江苏省“能源系统过程转化与减排技术”工程实验室 biography:. answer: The speaker for the lecture on Numerical Characterization of Powder Flowability is Dr. Pei Chunlei from the University of Cambridge's Department of Materials Science and Metallurgy.
question: What are some of the popular locations for Mandolin lessons near Hazelwood, MO? context: hazelwood, mo mandolin lessons looking for private in-home or in-studio mandolin hazelwood, mo check out some of our most popular teachers near hazelwood, mo including maryland heights, florissant, bridgeton, earth city and saint ann. brandon f.close _15<< · mandolin · in home brandon f. brandon f. about leanna f.close _

In [8]:
model_T5_pretrained.eval()
inputs = tokenizer_T5(promt, return_tensors="pt", max_length=2048, truncation=True)
inputs = {k: v.to(device) for k, v in inputs.items()}
with torch.no_grad():
    outputs = model_T5_pretrained.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        early_stopping=True
    )
gen_answer_1_shot=tokenizer_T5.decode(outputs[0], skip_special_tokens=True).strip()
gen_answer_zero_shot=generate_answer(model_T5_pretrained, tokenizer_T5, question=test_sample['question'], device=device, max_input_length=512, max_output_length=64, context=test_sample['context'])

### Let's compare the generated answer with the real one

In [9]:
print(f"Question: {test_sample['question']}\nGenereated one shot answer:{gen_answer_1_shot}\nGenereated zero shot answer:{gen_answer_zero_shot}\nDataset Answer:{test_sample['answer']}")
print(f"Comparison with real answer \nF1:{f1_score(gen_answer_1_shot,test_sample['answer']):.2f} BLEU:{bleu_score(gen_answer_1_shot,test_sample['answer']):.2f} EM:{exact_match_score(gen_answer_1_shot,test_sample['answer']):.2f}")

Question: What are some of the popular locations for Mandolin lessons near Hazelwood, MO?
Genereated one shot answer:maryland heights, florissant, bridgeton, earth city and saint ann
Genereated zero shot answer:maryland heights, florissant, bridgeton, earth city and saint ann
Dataset Answer:Some of the popular locations for Mandolin lessons near Hazelwood, MO are Maryland Heights, Florissant, Bridgeton, Earth City and Saint Ann.
Comparison with real answer 
F1:0.62 BLEU:0.31 EM:0.00


#### Aggregate Performance of zero shot calculated on test data (15% all data)

In [10]:
#avg_bleu, avg_em, avg_f1=eval_answers(model_T5_pretrained, tokenizer_T5,test_data,max_input=512,max_output=64)
print_scores(model_name="t5_zero_shot",use_backup=True)

Scores of t5_zero_shot model :
Average BLEU score: 2.69
Exact Match: 0.30%
F1 Score: 19.71%


## 5. Zero-Shot Evaluation Without Context

This step evaluates the model in a context-free setting to understand how much background information contributes to performance.


In [11]:
#avg_bleu, avg_em, avg_f1=eval_answers(model_T5_pretrained,tokenizer_T5,test_data,no_context=True)
print_scores(model_name="t5_zero_shot_no_context",use_backup=True)

Scores of t5_zero_shot_no_context model :
Average BLEU score: 1.53
Exact Match: 0.00%
F1 Score: 15.93%


## 6. Model Fine-Tuning

We fine-tune the pre-trained T5-small model on our QA dataset. This helps the model adapt to the specific language and structure of the task.

In [12]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq

tokenizer_T5_tuned = T5Tokenizer.from_pretrained("t5-small")
model_T5_tuned = T5ForConditionalGeneration.from_pretrained("t5-small")
model_T5_tuned.to(device)
train,val = train_test_split(finetuning_data, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train)
val_dataset = Dataset.from_pandas(val)
tokenizer=tokenizer_T5_tuned
train_dataset = train_dataset.map(preprocess,batched=False,fn_kwargs={
        "tokenizer":tokenizer_T5_tuned,
        "max_length_input": 512,
        "max_length_labels": 64
    })
val_dataset = val_dataset.map(preprocess,batched=False,fn_kwargs={
        "tokenizer":tokenizer_T5_tuned,
        "max_length_input": 512,
        "max_length_labels": 64
    })
training_args = TrainingArguments(
    output_dir="/kaggle/working",
    do_train=True,
    do_eval=True,
    learning_rate=0.000472,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=6,
    weight_decay=0.041494,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=1000,     
    eval_steps=1000,      
    save_total_limit=2,
    report_to="none",
)
trainer = Trainer(
    model=model_T5_tuned,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer_T5_tuned
)
trainer.train()

Map:   0%|          | 0/6124 [00:00<?, ? examples/s]

Map:   0%|          | 0/1531 [00:00<?, ? examples/s]

/tmp/ipykernel_35/2746039429.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,1.006500
1000,0.893800
1500,0.836000
2000,0.720900
2500,0.701300
3000,0.706500
3500,0.612800
4000,0.611600
4500,0.619600
5000,0.527000


TrainOutput(global_step=9186, training_loss=0.6053531737258018, metrics={'train_runtime': 996.5277, 'train_samples_per_second': 36.872, 'train_steps_per_second': 9.218, 'total_flos': 4972999153287168.0, 'train_loss': 0.6053531737258018, 'epoch': 6.0})

In [18]:
#save 
trainer.save_model("/kaggle/working/trainer")
tokenizer_T5_tuned.save_pretrained("/kaggle/working/tokenizer")

('/kaggle/working/tokenizer/tokenizer_config.json',
 '/kaggle/working/tokenizer/special_tokens_map.json',
 '/kaggle/working/tokenizer/spiece.model',
 '/kaggle/working/tokenizer/added_tokens.json')

In [14]:
#load
tokenizer_T5_tuned = T5Tokenizer.from_pretrained("/kaggle/input/trained-nlp-models/tokenizer_t5/tokenizer")
model_T5_tuned = T5ForConditionalGeneration.from_pretrained("/kaggle/input/trained-nlp-models/model_t5/trainer")
model_T5_tuned.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

### Evaluation: Fine-Tuned Model

In [15]:
#avg_bleu, avg_em, avg_f1=eval_answers(model_T5_tuned,tokenizer_T5_tuned,test_data,max_input=512,max_output=64)
print_scores(model_name="t5_tuned",use_backup=True)

Scores of t5_tuned model :
Average BLEU score: 39.32
Exact Match: 10.11%
F1 Score: 59.45%


### let's test the previus answer

In [16]:
tuned_model_answer=generate_answer(model_T5_tuned, tokenizer_T5_tuned, question=sample_zero_shot["question"].values[0], device=device, max_input_length=512, max_output_length=64, context=sample_zero_shot["context"].values[0])
sample_zero_shot["question"].values[0],tuned_model_answer, sample_zero_shot["answer"].values[0]

('What are the responsibilities and requirements for the Software Architect position at the entertainment company?',
 'The responsibilities and requirements for the Software Architect position at the entertainment company include developing and analysing the design and architecture of complex software application systems, providing architectural and implementation oversight and guidance to ensure consistency and quality of design and code, analyzing and documenting existing systems, reviewing preexisting complex code',
 'The responsibilities and requirements for the Software Architect position include developing and analyzing the design and architecture of complex software application systems, providing architectural and implementation oversight, analyzing and documenting existing systems, communicating technical issues clearly, writing and debugging complex problems in various software modules, managing test and acceptance activities, designing and building reusable modules, writing a

## 7. Hyperparameter Tuning on 15% of the Dataset

In order to optimize the T5 model performance, we performed a hyperparameter tuning process using only 15% of the training dataset. 

The following table summarizes the results of 27 different trials. Each trial is evaluated using a composite value score, which considers BLEU, EM, and F1. The best performing configurations are those with the highest `value`.

Hyperparameters tuned:
- **Learning Rate (lr)**: Controls how fast the model adapts during training.
- **Batch Size**: Affects stability and memory usage during training.
- **Epochs**: Number of times the model sees the entire dataset.
- **Learning Rate Scheduler (lr_sched)**: Strategy to adjust the learning rate over time.
- **Weight Decay**: Regularization term to prevent overfitting.

### Results 
Below is the table with trial configurations and their corresponding scores:

In [17]:
import optuna 
study = optuna.load_study(
        study_name='t5_small',
        storage=f'sqlite:////kaggle/input/nlp-resources/t5_small_hyper_parameter_tuning.db')
import pandas as pd
data = []
for trial in study.trials:
    trial_data = trial.params.copy()
    trial_data.update(trial.user_attrs)
    trial_data["value"] = trial.value  # score o loss principale
    trial_data["trial_number"] = trial.number
    data.append(trial_data)

trials_df = pd.DataFrame(data)[:27]
trials_df


,lr,batch,epochs,lr_sched,weight_decay,BLEU,EM,F1,value,trial_number
0,0.000011,8,2,linear,0.009554,4.062378,1.096892,6.564946,4.674273,0
1,0.000116,4,5,cosine,0.035746,40.541588,10.847044,61.165567,44.007612,1
2,0.000012,8,6,linear,0.014758,34.300564,7.800122,55.112598,38.837651,2
3,0.000417,4,2,linear,0.049547,39.362632,10.115783,59.778431,42.838056,3
4,0.000014,4,4,cosine,0.034493,34.996220,8.409506,55.623600,39.396634,4
5,0.000161,4,6,cosine,0.037204,40.901913,11.151737,61.257725,44.190348,5
6,0.000198,8,3,linear,0.030070,40.097911,11.090798,60.592315,43.692420,6
7,0.000041,8,5,linear,0.027798,37.740979,9.567337,58.488850,41.737609,7
8,0.000026,8,5,cosine,0.001707,36.188469,9.140768,56.976599,40.547037,8
9,0.000182,8,2,linear,0.011389,38.367603,10.176722,59.046327,42.317573,9


## 8. Final Comparison of All Model Configurations

This table summarizes the performance of all major configurations evaluated throughout the project.

| Model Configuration         | Context Used | Training Type     |On all test data |Exact Match (%) | F1 Score (%) | BLEU Score |
|----------------------------|--------------|-------------------|-----------------|--------------|------------|------------|
| Zero-Shot                  | yes           | Pretrained only   | True            | 0.30            | 19.71         | 2.79       |
| Zero-Shot (No Context)     | no            | Pretrained only   | True            | 0.00            | 15.93         | 1.53       |
| Few-Shot (1 example)       | yes           | Prompting only    | False (1 sample)            | 0.00            | 62.0         | 31       |
| Fine-Tuned (Best Config)   | yes           | Supervised tuning | True            | 10.11           | 59.45         | 39.32       |
